# **Bootcamp Data Analytics - Squad Jaqueline Goes - Desafio Gerda**u
*   Camila Cestari Cerreti
*   Carla Mendes da Silva
*   Cláudia dos Santos Silva
*   Leticia Shiota
*   Lizandra Broseghini Föeger
*   Nara Guimarães





**Instruções:**
Descreva a abordagem utilizada na resolução.\
Trechos de códigos deverão ter comentários essenciais.\
Haverá sessão de dúvidas, direcionando o entendimento dos problemas. Não haverá comentário de solução.


## **1) Estudo do desempenho da siderurgia brasileira**

A indústria do aço possui grande importância na indústria de transformação brasileira, na participação do PIB e na geração de empregos. Segundo o IABr (Instituto Aço Brasil) a produção de aço bruto brasileiro atingiu 32,6 milhões de toneladas em 2019, colocando o Brasil como o 9º maior produtor de aço no mundo. Foram 112 mil colaboradores, entre próprios e terceiros, que tornaram esse resultado possível.

[https://acobrasil.org.br/](https://acobrasil.org.br/)



## Tratamento de dados

In [19]:
# Importação de bibliotecas
import pandas as pd

In [25]:
# Importação de dados
performance = pd.read_excel('../Projeto Gerdau/Performance_Mensal.xls', header = 4)
performance

FileNotFoundError: ignored

In [ ]:
# Verificando o número de colunas
performance.shape[1]

In [ ]:
# Verificando quais são as colunas
performance.columns

In [ ]:
# Arrumando a base de dados
## Nomeando as colunas
def months_columns (mes_inicio, ano_inicio, n_coluna):
    nome_col = []
    lista_mes = ['Jan', 'Fev', 'Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
    i = 0 
    while(i<n_coluna):
        nome_temp = lista_mes[mes_inicio -1] +"_" +str(ano_inicio)
        nome_col.append(nome_temp)
        mes_inicio += 1
        if mes_inicio == 13:
            mes_inicio = 1
            ano_inicio += 1
        i +=1
    return nome_col

def fix_df(df, mes_inicio, ano_inicio, n_coluna=98):
    months_list = months_columns(mes_inicio, ano_inicio, n_coluna)
    new_columns = list()
    new_columns.append('especificacao')
    new_columns.extend(months_list)
    new_columns.append('pct')
    df.columns = new_columns
    return df

In [ ]:
# Aplicando a função
temp = fix_df(df=performance, mes_inicio=1,ano_inicio=2013)

In [ ]:
# Analisando a base de dados
temp

In [ ]:
# Removendo as linhas que não serão utilizadas e estão com valores nulos.
temp2 =pd.DataFrame(temp.drop([0,44,45,46,47,48,49], axis=0))
temp2

- Produção: Aço Bruto + Laminados (Planos + Longos) + Semiacabado p/ venda (Placas + Blocos e Tarugos) + Ferro-Gusa;
- Venda internas: Laminados (Planos + Longos) + Semiacabado p/ venda (Placas + Blocos e Tarugos);
- Venda ezxterna: Laminados (Planos + Longos) + Semiacabado p/ venda (Placas + Blocos e Tarugos);
- Comércio Exterior / Exportação: Laminados (Planos + Longos) + Semiacabado p/ venda (Placas + Blocos e Tarugos);
- Importações: Laminados (Planos + Longos) + Semiacabado p/ venda
- Laminados: Planos + Longos;
- Semiacabado p/ venda: Placas + Blocos e Tarugos;
- Consumno aparente: Planos + Longos.

Considerando as linhas acima como sendo o resultado da somatória das demais linhas. As linhas produção, venda interna, venda externa, comércio exterior, exportações, importações, e consumno aparente, laminados, semiacabados p/venda serão removidos com exceção da linha semiacado p/ Venda pois esta não apresenta uma subcategoria.

A linhas Total e Milhões US$ das categorias de importação e exportação serão incluídas em outro DataFrame.


In [ ]:
temp3 =pd.DataFrame(temp2.drop([1,3,6,10,11,14,17,18,21,24,25,26,29,32,33,34,35,39,40,41,], axis=0))
temp3

In [ ]:
# Reset index
temp3.reset_index(drop=True, inplace = True)

In [ ]:
# Renomeando as linhas da coluna 'especificação' para identificar a categoria que elas pertencem, sendo:
# Produção
# Venda interna - Vinterna
# Venda externa - Vexterna
# Comércio Exterior/Exportação - Cexterior
# Importação
# Consumo Aparente - Caparente
value = ["Producao.Aco Bruto", "Producao.Planos","Producao.Longos","Producao.Placas","Producao.Blocos","Producao.Ferro-Gusa","Vinterna.Planos", 
         "Vinterna.Longos","Vinterna.Placas","Vinterna.Blocos","Vexterna.Planos","Vexterna.Longos","Vexterna.Placas","Vexterna.Blocos", 
         "Cexterior.Planos","Cexterior.Longos","Cexterior.Placas","Cexterior.Blocos","Importacao.Planos","Importacao.Longos", 
         "Importacao.Semiacabados p/ Venda","Caparente.Planos", "Caparente.Longos"]
temp3.iloc[:,[0]] = value
temp3

In [ ]:
# Tranformando os dados do formato largo para o formato longo.
lista_ano = temp3.columns
lista_ano = lista_ano[1:99]
df = pd.melt(temp3, id_vars=['especificacao'], value_vars=lista_ano)
df.rename(columns = {'variable':'periodo', 'value':'quantidade'}, inplace = True)
df

In [ ]:
# Criando uma coluna para ano e uma coluna para mês
mes = []
ano = []
for per in df['periodo']:
    periodo = per.split('_')
    mes.append(periodo[0])
    ano.append(periodo[1])   
df['ano'] = ano
df['mes'] = mes
df

In [ ]:
# Criando uma coluna para subcategoria e uma coluna para a categoria
categoria = []
subcategoria = []
for esp in df['especificacao']:
    total = esp.split('.')
    categoria.append(total[0])
    subcategoria.append(total[1])   
df['categoria'] = categoria
df['subcategoria'] = subcategoria
df

In [ ]:
# Criando uma coluna para o trimestre 
dict_tri = {
    'Jan':1,
    'Fev':1,
    'Mar':1,
    'Abr':2,
    'Mai':2,
    'Jun':2,
    'Jul':3,
    'Ago':3,
    'Set':3,
    'Out':4,
    'Nov':4,
    'Dez':4,
}
df['trimestre'] = df['mes'].map(dict_tri)
df

In [ ]:
# Reordenando as colunas
df = df[['especificacao', 'categoria', 'subcategoria','trimestre','periodo','ano','mes','quantidade']]
df

In [ ]:
# Salvando o DataFrame 
df.to_csv('C:\\Users\\leshi\\Projeto_Gerdau\\Quantidade_subcategoria.csv', index=False)

In [ ]:
# Criando o DataFrame para Importacao e Exportacao
df_imp = temp2.iloc[[31,32,38,39]]
df_imp.head()

In [ ]:
# Renomeando as linhas
row = ["Exportacao.Total(mil_t)", "Exportacao.milhoes(dolar)","Importacao.Total(mil_t)","Importacao.milhoes(dolar)"]
df_imp.loc[0:4,['especificacao']] = row
df_imp

In [ ]:
# Tranformando os dados do formato largo para o formato longo.
df_imp_exp = pd.melt(df_imp, id_vars=['especificacao'], value_vars=lista_ano)
df_imp_exp.rename(columns = {'variable':'periodo', 'value':'valor'}, inplace = True)
df_imp_exp

In [ ]:
#Criando uma coluna para ano, mês, categoria e subcategoria
mes = []
ano = []
categoria =[]
subcategoria = []
for per in df_imp_exp['periodo']:
    periodo = per.split('_')
    mes.append(periodo[0])
    ano.append(periodo[1])   
df_imp_exp['ano'] = ano
df_imp_exp['mes'] = mes
for cat in df_imp_exp['especificacao']:
    conjunto = cat.split('.')
    categoria.append(conjunto[0])
    subcategoria.append(conjunto[1])   
df_imp_exp['categoria'] = categoria
df_imp_exp['subcategoria'] = subcategoria
df_imp_exp

In [ ]:
# Criando uma coluna para o trimestre
dict_imp = {
    'Jan':1,
    'Fev':1,
    'Mar':1,
    'Abr':2,
    'Mai':2,
    'Jun':2,
    'Jul':3,
    'Ago':3,
    'Set':3,
    'Out':4,
    'Nov':4,
    'Dez':4
}
df_imp_exp['trimestre'] = df_imp_exp['mes'].map(dict_imp)

In [ ]:
# Reordenando as colunas
df_imp_exp = df_imp_exp[['especificacao','categoria','subcategoria','trimestre','periodo','ano','mes','valor']]
df_imp_exp

In [ ]:
# Salvando o DataFrame importação e exportação
df_imp_exp.to_csv('Importacao_exportacao.csv', index=False)

## Desenvolvimento item 1

Os dados da planilha Performance_Mensal foram formatados conforme a seção Tratamento de dados, e os dados foram dispostos em dois Data Frames:

df_quantidade: Dataframe demonstrando as quantidade das subcategorias por período.

df_imp_exp: Quantidade total de aço importado e exportado e seus respectivos valores.


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from prophet import Prophet
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [21]:
# Importando o Dataframe df_quantidade
df_quantidade = pd.read_csv('Quantidade_subcategoria.csv',sep=',')
df_quantidade

ParserError: ignored

In [23]:
# Importando o Dataframe df_imp_exp
df_imp_exp =  pd.read_csv('../dados/Importacao_exportacao.csv',sep=',')
df_imp_exp

FileNotFoundError: ignored

**Utilizando os dados históricos disponíveis no arquivo de excel "Performance_Mensal.xls" responda as seguintes pergutas e descreva o raciocínio utilizado.**


**a. As exportações de aço brasileiras são representativas no mercado internacional, o Brasil foi o 12º maior exportador de produtos siderúrgicos em 2019. Qual foi o trimestre em que houve a maior exportação de aço? Qual foi o preço aproximado? É possivel identificar alguma relação entre preço e quantidade exportada?**

In [ ]:
# Trimestre com maior exportação de aço em 2019
df_fil = df_imp_exp[(df_imp_exp.ano == 2019) & (df_imp_exp.categoria =='Exportacao') 
                    & (df_imp_exp.subcategoria=='Total(mil_t)')]

df_tri = pd.DataFrame(df_fil.groupby(['trimestre'])['valor'].sum())

df_tri = df_tri.sort_values(by='valor',ascending=False)
print('O segundo trimestre teve maior número de exportação com:'+ ' ' + str(df_tri.iloc[0][0]) + ' ' + 'toneladas')

O segundo trimestre teve maior número de exportação com: 3398.041 toneladas.

In [ ]:
# Preço médio aproximado do aço em dolar/tonelada:
df_avg = df_imp_exp[(df_imp_exp.ano ==2019)&(df_imp_exp.categoria =='Exportacao')&(df_imp_exp.trimestre ==2)]

df_avg = pd.DataFrame(df_avg.groupby(['subcategoria'])['valor'].sum())

avg = ((df_avg.iloc[1]*1000000) / (df_avg.iloc[0]*1000)).round(3)
print('A média do preço do aço no segundo trimestre de 2019 foi:' + ' ' 'US$'+ str(avg[0]) + ' '+'por tonelada.')

A média do preço do aço no segundo trimestre de 2019 foi: US$606.699 por tonelada.

In [ ]:
# Analisando a correlação entre quantidade exportada e preço médio
table = df_imp_exp.pivot_table('valor', index =['categoria', 'periodo', 'ano', 'mes','trimestre'], columns =['subcategoria'])

table_ren = table.rename_axis(None,axis=1)

table_ren = table_ren.reset_index()
table_ren['preco_medio(dolar/ton)'] = (table_ren['milhoes(dolar)']*1000000) / (table_ren['Total(mil_t)']*1000)
table_ren[['preco_medio(dolar/ton)','Total(mil_t)']].corr()

In [ ]:
# Plotando a correlação entre preço médio e quantidade
ax = sns.scatterplot(data = table_ren, x="preco_medio(dolar/ton)", y="Total(mil_t)")
ax.set_title('Correlação entre preço médio e quantidade', size = 13)
ax.set_xlabel('Preço_médio(US$/ton)')
ax.set_ylabel('Quantidade(Mil_ton)')
ax;

Conclusão: No ano de 2019, o periodo com maior exportação foi o segundo trimestre com 3398.041 tonelas com o preço médio de US$606.699 por tonelada. Avaliando a correlação entre a quantidade exportada eo preço médio é possivel obervar que há uma correlação negativa exponencial entre as variáveis.


**b. Dado que o desempenho da siderurgia está ligado ao desempenho econômico do país, utilize a série histórica do consumo aparente de aço brasileiro e faça uma análise exploratória, explicando as mudanças de tendência observadas.**

In [ ]:
# Avaliando graficamente o consumo aparente ao longo do tempo
df_consumo = df_quantidade[df_quantidade['categoria']=='Caparente']
aparente = df_consumo.groupby(df_consumo['periodo'])['quantidade'].sum()

plt.figure(figsize = (12,6))
ap = aparente.plot()
ap.set_title('Consumo aparente de aço',size=15)
ap.set_ylabel('Quantidade(mil_ton)')
ap.set_xlabel('Período')
ap;

A análise exploratória utilizará a taxa de variação percentual do consumo aparente, onde a taxa de variação será calculado em relação ao mesmo trimestre do ano anterior. O período presente na planilha Performance_mensal é de Jan/2013 a Fev/2021, para realizar a análise os dados de Jan e Fev 2021 serão desconsiderados uma vez que não há dado suficiente do primeiro trimestre de 2021. Adicionalmente, não será possível calcular a taxa de variação de 2013 uma vez que não temos os dados de 2012.

In [ ]:
# Adicionando a coluna de taxa de variação(%)
df_total = pd.DataFrame(df_consumo.groupby(['ano','trimestre'])['quantidade'].sum()).reset_index()
df_total.sort_values(by=['trimestre', 'ano'] , inplace = True)
df_ca= df_total.pivot_table('quantidade',index =['trimestre'], columns =['ano'])
df_ca = df_ca.rename_axis(None, axis=1)  
df_ca = df_ca.reset_index()
df_ca

In [ ]:
# Adicionando a coluna de taxa de variação(%)
x = 1
b = 0
while(x< len(df_ca.columns)-1 and b in range(0,7)):     
        lista_ano = [2014,2015,2016,2017,2018,2019,2020,2021]
        df_ca['var'+ '_'+ str(lista_ano[b])] = (df_ca[df_ca.columns[x+1]]/df_ca[df_ca.columns[x]]-1)*100
        b += 1
        x += 1
df_ca 

In [ ]:
# Dropando as colunas ano
df_var = df_ca.drop(columns = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
df_var

In [ ]:
# Transformando o dado de largo para longo:
df_var.rename({'var_2014':2014, 'var_2015':2015, 'var_2016':2016, 'var_2017':2017, 
               'var_2018':2018, 'var_2019':2019, 'var_2020':2020}, axis = 1, inplace=True)

lista = df_var.columns
lista = lista[1:8]
df_bd = pd.melt(df_var, id_vars=['trimestre'], value_vars=lista, var_name='ano', value_name='c_aparente')
df_bd

In [ ]:
# Importando os dados do IBGE
df_pib = pd.read_excel('../dados/Variacao_PIB.xlsx')
df_pib

In [ ]:
# Juntando os DataFrames:
data = pd.merge(df_bd, df_pib, how="inner", on=["trimestre", "ano"])
data

In [ ]:
# Analisando as variaveis
pd_all = pd.melt(data, id_vars=['ano_tri'], value_vars=['c_aparente','PIB_pm'], 
                 var_name='variavel', value_name='taxa_variacao')

plt.figure(figsize = (15,8))
ax = sns.lineplot(data=pd_all, x="ano_tri", y="taxa_variacao", hue="variavel")
plt.xticks(rotation=30);



O PIB apresenta uma forte relação com o consumo aparente do aço. Houve duas quedas acentuadas do consumo do aço ao longo do período. Uma queda que se inciou em 2015 e começou a se recuperar em 2016 e outra queda no início de 2020.

Em 2015 o Brasil enfrentou uma crise econômica que causou uma quedo no PIB de 3,2% entre os meses de Janeiro a Setembro adicionalmente, o país estava vivendo uma instabilidade política com os escandelos de corrupção da Petrobras e Lava-Jato e abertura do processo de Impeachment de Dilma Rousseff. Esses fatores podem ter contribuído para a queda do consumo do aço. No final de 2019 e início do ano de 2020 houve o início da pandemia do Covid-19 o que pode ter causado a queda no consumo aparente do aço.


**c. Projeções são importantes para avaliar situações futuras e realizar planejamentos. Faça uma projeção do consumo aparente de aço para os próximos 3 anos (2020 - 2022) e avalie quais fatores e/ou variáveis influenciam a projeção.**

In [ ]:
# Vendo o dataframe quantidade
df_quantidade.head()

In [ ]:
# Filtrando a categoria 'Caparente' (Consumo aparente), criando a coluna 'ds' e renomeando a coluna 'quantidade'
df_forecast = df_quantidade[df_quantidade['categoria']=='Caparente'].groupby(['categoria','periodo'])['quantidade'].sum().reset_index()
mes = []
ano = []
for per in df_forecast['periodo']:
    periodo = per.split('_')
    mes.append(periodo[0])
    ano.append(periodo[1])   
df_forecast['ano'] = ano
df_forecast['mes'] = mes

map_mes = {
    "Jan": 1,
    "Fev": 2,
    "Mar": 3,
    "Abr": 4,
    "Mai": 5,
    "Jun": 6,
    "Jul": 7,
    "Ago": 8,
    "Set": 9,
    "Out": 10,
    "Nov": 11,
    "Dez": 12
}
df_forecast['mes_num'] = df_forecast['mes'].map(map_mes)
df_forecast['ds'] = df_forecast['ano'].astype(str)+'-'+df_forecast['mes_num'].astype(str)
df_forecast = df_forecast[['ds', 'quantidade']]
df_forecast['ds'] =  pd.to_datetime(df_forecast['ds'], format='%Y-%m')
df_forecast.rename({'quantidade': 'y'}, axis=1, inplace=True)
df_forecast = df_forecast.sort_values(by="ds")

In [ ]:
# Analisando o dataframe forecast
df_forecast.tail()

In [ ]:
# Utilizando a bibloteca Prophet para realizar o forecast 
m = Prophet().fit(df_forecast)
future = m.make_future_dataframe(periods=34, freq='MS')
fcst = m.predict(future)
fig = m.plot(fcst)

In [ ]:
# Dataframe do previsto com a data, média, intervalo de confiança inferior e intervalo de confiança superior
df_previsto = fcst[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
df_previsto.tail()

In [ ]:
# Realizando a decomposicao
fig2 = m.plot_components(fcst)

In [ ]:
# Crossvalidation 
from prophet.diagnostics import cross_validation
cutoffs = pd.date_range(start='2018-01-01', end='2020-01-01', freq='12MS')
df_cv = cross_validation(model=m, horizon='365 days', cutoffs=cutoffs)

In [ ]:
# Calculando as metricas da crossvalidation
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head(10)

Conclusão: a previsão mostra um crescimento do consumo aparente do aço para os anos de 2021 a 2023. O modelo parece bem ajustado para o período que foi treinado. Os pontos que houve uma variação muito alta do consumo, o modelo não foi capaz de se ajustar, porém o modelo obteve um baixo erro percentual absoluto médio calculado pelo crossvalidation demonstrando uma boa acurácia.

## **2) Conhecimento geral**

Baseado nos dados (carteira_demanda.xls) fornecidos da carteira da GSP do dia 25 de março, construa os seguintes gráficos com as bibliotecas da sua preferência (seaborn, matplotlib, Altair, etc). Tente se aproximar o máximo possível a descrição fornecida na imagem
Carteira por tipo de demanda

A GSP atende as seguintes demanda:

    Comercial Gerdau
    Corte e Dobra
    Produção para Estoque
    Mercado Externo
    Mercado Interno
    Planta/Transferência

Precisamos criar um gráfico que mostre os volumes da nossa carteira para cada mês e para cada tipo de demanda, além de ter a visão se a carteira foi predita ou ela já está implantada.

Faça uma breve descrição do gráfico e explique os filtros/tratamento nos dados.




## Desenvolvimento item 2

In [24]:
# Importando a base de dados
df_carteira = pd.read_excel('../dados/carteira_demanda.xls')
df_carteira

FileNotFoundError: ignored

In [ ]:
# Selecionando as colunas que serão utilizadas
df_gps = df_carteira[['demand_type', 'due_month','predicted','quantity']]
df_gps

In [ ]:
# Verificando se há valores nulos
df_gps.isnull().values.any()

In [ ]:
# Agrupando os dados por tipo de demanda, mês e previsão
df_carteira_gps = pd.DataFrame(df_gps.groupby(['demand_type', 'due_month','predicted'])['quantity'].sum()).reset_index()
df_carteira_gps

In [ ]:
# Plotando o gráfico para os meses M0 a M5
temp = df_carteira_gps[df_carteira_gps['due_month'].isin(['M0','M1','M2','M3','M4','M5'])]
temp.rename({'predicted':'Previsão'},axis=1,inplace=True)

(ggplot(temp, aes(x='demand_type', y='quantity', fill='Previsão'))
 + geom_bar(stat='identity', position='stack')
 + facet_wrap('due_month', ncol=6)
 + theme(figure_size = (20,6), axis_text_x=element_text(rotation=90, hjust=1))
 + labs(title ='Carteira da GPS por tipo de demanda', y = "Volume Total[ton]", x="Tipo de demanda")
)

Conclusão: O gráfico demonstra que para os meses M3, M4 e M5 a carteira foi predominantemente preditiva.